In [1]:
import pandas as pd
import glob
import chardet
from csv import reader
from io import StringIO

In [2]:
def detect_encoding(file_path, sample_size=1024):
    """Detect file encoding with chardet, using only a sample of the file"""
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read(sample_size))
    return result['encoding']

def robust_csv_read(file_path, fallback_encodings=['utf-8', 'latin1', 'cp1252']):
    """Try multiple methods to read a problematic CSV file"""
    # First try pandas with auto-detected encoding
    try:
        encoding = detect_encoding(file_path)
        return pd.read_csv(file_path, encoding=encoding, on_bad_lines='warn')
    except Exception as e:
        print(f"Primary read failed for {file_path}, trying fallbacks... Error: {str(e)[:100]}")

    # Try fallback encodings
    for enc in fallback_encodings:
        try:
            return pd.read_csv(file_path, encoding=enc, on_bad_lines='warn')
        except:
            continue

    # If all else fails, use manual CSV reader
    try:
        with open(file_path, 'r', encoding='latin1') as f:
            content = f.read()
        
        # Use Python's CSV reader to handle problematic lines
        csv_reader = reader(StringIO(content))
        rows = list(csv_reader)
        
        # Find max columns and pad rows
        max_cols = max(len(row) for row in rows)
        padded_rows = [row + [None]*(max_cols - len(row)) for row in rows]
        
        # Create DataFrame (first row as header)
        return pd.DataFrame(padded_rows[1:], columns=padded_rows[0])
    except Exception as e:
        print(f"Completely failed to read {file_path}: {e}")
        return None

def concatenate_csv_files(file_pattern, output_file=None):
    """Main function to concatenate multiple CSV files"""
    csv_files = glob.glob(file_pattern)
    if not csv_files:
        raise ValueError(f"No files found matching pattern: {file_pattern}")

    dataframes = []
    for file in csv_files:
        df = robust_csv_read(file)
        if df is not None:
            dataframes.append(df)
            print(f"Successfully read {file} with shape {df.shape}")
        else:
            print(f"Failed to read {file}")

    if not dataframes:
        raise ValueError("No valid DataFrames were created from the input files")

    combined_df = pd.concat(dataframes, ignore_index=True)
    
    if output_file:
        combined_df.to_csv(output_file, index=False)
        print(f"Saved combined data to {output_file}")
    
    return combined_df


In [4]:
try:
    result_df = concatenate_csv_files(
        file_pattern='C:/Users/KseniaLuschevaext/Documents/thesis/newtext/*.csv',
        output_file='newtext.csv'
        )
    print(f"Final combined DataFrame shape: {result_df.shape}")
except Exception as e:
    print(f"Error in concatenation process: {e}")

Successfully read C:/Users/KseniaLuschevaext/Documents/thesis/newtext\gends2.csv with shape (629, 9)
Successfully read C:/Users/KseniaLuschevaext/Documents/thesis/newtext\gen_ds.csv with shape (498, 9)
Successfully read C:/Users/KseniaLuschevaext/Documents/thesis/newtext\gen_ds3.csv with shape (988, 9)
Successfully read C:/Users/KseniaLuschevaext/Documents/thesis/newtext\gen_yagpt_03.csv with shape (820, 9)
Saved combined data to newtext.csv
Final combined DataFrame shape: (2935, 9)


In [12]:
df_all = pd.read_csv('newtext.csv')

In [13]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935 entries, 0 to 2934
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             2935 non-null   object
 1   prompt           2935 non-null   object
 2   style            2935 non-null   object
 3   topic            2115 non-null   object
 4   model            2935 non-null   object
 5   length           2935 non-null   int64 
 6   valid            2935 non-null   bool  
 7   generation_date  2935 non-null   object
 8   word_count       2935 non-null   int64 
dtypes: bool(1), int64(2), object(6)
memory usage: 186.4+ KB


In [19]:
df_all.drop(columns=['generation_date','style','topic','word_count','valid','prompt'], inplace=True)

In [33]:
df_all['text'] = df_all['text'].str.split('Рассмотрим вопрос').str[-1]

In [34]:
df_all.iloc[100,0]

' письма будущего: общение сквозь века. Каким станет диалог между поколениями, когда технологии сотрут границы времени? Уже сегодня мы стоим на пороге эпохи, где слова, запечатлённые в цифре или даже в памяти вселенной, смогут путешествовать сквозь столетия, соединяя сердца и умы людей из разных эпох. Это не просто мечта — это ближайшая реальность, и она восхищает!  \n\n### **Время больше не преграда**  \nПредставьте: ваше послание, наполненное эмоциями, мыслями или даже голографическим образом, достигает потомков через сто, двести лет. Они читают ваши строки, слышат ваш голос, видят вашу улыбку — и время словно исчезает. Технологии блокчейна гарантируют подлинность текста, квантовые носители сохранят данные тысячелетиями, а искусственный интеллект поможет адаптировать язык так, чтобы он оставался понятным даже через века. Письмо перестаёт быть статичным документом — оно становится живым мостом между эпохами.  \n\nНо что ещё важнее — такой формат общения изменит само восприятие истории

In [36]:
dataframes = []
x = pd.read_csv('/Users/klushcheva/Documents/thesis/gen_texts/2.csv')
dataframes.append(x)
y = pd.read_csv('/Users/klushcheva/Documents/thesis/gen_texts/3.csv')
dataframes.append(y)

df2 = pd.concat(dataframes, ignore_index=True)

In [37]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   text             21 non-null     object 
 1   prompt           21 non-null     object 
 2   style            21 non-null     object 
 3   topic            1 non-null      object 
 4   model            21 non-null     object 
 5   length           21 non-null     float64
 6   valid            21 non-null     bool   
 7   reason           0 non-null      float64
 8   generation_date  21 non-null     object 
 9   word_count       21 non-null     int64  
dtypes: bool(1), float64(2), int64(1), object(6)
memory usage: 1.6+ KB


In [38]:
df2.drop(columns=['generation_date','style','topic','word_count','valid','prompt','reason'])

,text,model,length
0,"""Сэр Джеймс"" (The Lord James), перевод А. Ивин...",ai-forever/rugpt3large_based_on_gpt2,429.0
1,{{name}}\tmaggot_ru \/fontsize=24px|bel-bright...,ai-forever/rugpt3large_based_on_gpt2,409.0
2,/*Примечание редактора */\n @font-face {\n...,ai-forever/rugpt3large_based_on_gpt2,339.0
3,[ИТОГИ СТИЛЯ ДИАЛога.] \n Автор (автор)...,ai-forever/rugpt3large_based_on_gpt2,426.0
4,"""Я не могу объяснить вам почему я это делаю"". ...",ai-forever/rugpt3large_based_on_gpt2,387.0
5,Если ты новичок или работаешь над новым проект...,ai-forever/rugpt3large_based_on_gpt2,397.0
6,[ОБЪЯСНЕНИЕ ТЕХНИКИ И ПЕРЕХОДА К УЧЕБЕ.] \n ...,ai-forever/rugpt3large_based_on_gpt2,452.0
7,[ПРАВИЛА ИХ ВЫПОЛНЕНИЯ.] \n 1-2 абзаца д...,ai-forever/rugpt3large_based_on_gpt2,361.0
8,1) Для начала изложения текста укажи тему бесе...,ai-forever/rugpt3large_based_on_gpt2,472.0
9,"{""name"": ""test"",""[0],[1].replace(/\s*/gi|utf-8...",ai-forever/rugpt3large_based_on_gpt2,348.0


In [53]:
df = pd.concat([df_all,df2], ignore_index=True)
df = df.drop(columns=['generation_date','style','topic','word_count','valid','prompt','reason','text;prompt;style;topic;model;length;valid;generation_date;word_count'])

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377 entries, 0 to 7376
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    6887 non-null   object 
 1   model   6887 non-null   object 
 2   length  6887 non-null   float64
dtypes: float64(1), object(2)
memory usage: 173.0+ KB


In [55]:
df1 = pd.read_csv('/Users/klushcheva/Documents/thesis/gen_texts/1.csv',on_bad_lines='warn')
df4 = pd.read_csv('/Users/klushcheva/Documents/thesis/gen_texts/4.csv',on_bad_lines='warn',sep=';')
df5 = pd.read_csv('/Users/klushcheva/Documents/thesis/gen_texts/5.csv',on_bad_lines='warn')

In [56]:
df1.drop(columns=['generation_date','word_count'],axis=1, inplace=True)

In [57]:
df4.drop(columns=['generation_date','word_count','valid','prompt','reason','word_count'],axis=1, inplace=True)
df4['model']='ai-forever/rugpt3small_based_on_gpt2'
df4.head()

,text,length,model
0,+ Название проекта (сначала)\n[ПОНЯТИЙ И СТАТЕ...,303,ai-forever/rugpt3small_based_on_gpt2
1,"||""Синтез ДНК"", ""Биохимия человека"".\t1-2 стра...",339,ai-forever/rugpt3small_based_on_gpt2
2,Веб-программирование для начинающих Авторский ...,303,ai-forever/rugpt3small_based_on_gpt2
3,"\nЕсть массив данных SQLite5, нужно вывести ег...",301,ai-forever/rugpt3small_based_on_gpt2
4,"\t""У каждого человека есть свой внутренний гол...",380,ai-forever/rugpt3small_based_on_gpt2


In [58]:
df5.drop(columns=['generation_date','word_count','valid','prompt','reason','style'],axis=1, inplace=True)
df5.head()

,text,topic,model,length
0,+/- 1 строка для текста длиной 200 символов (с...,NaN,ai-forever/rugpt3large_based_on_gpt2,320.0
1,Примечание 1\n В тексте не должно содержаться ...,NaN,ai-forever/rugpt3large_based_on_gpt2,327.0
2,Далее можешь написать о себе (название професс...,NaN,ai-forever/rugpt3large_based_on_gpt2,374.0
3,[ИТОГОВЫЙ РЕДАКСИОННЫЙ СТИЛЬ ОТДЕЛЕНИЙ АУДИО-Т...,NaN,ai-forever/rugpt3large_based_on_gpt2,347.0
4,В конце текста укажи ссылку на источник информ...,NaN,ai-forever/rugpt3large_based_on_gpt2,352.0


In [59]:
df_final = pd.concat([df,df1,df4,df5], ignore_index=True)

In [61]:
df_final.drop(columns=['prompt','topic'], inplace=True, axis=1)

In [44]:
df_trim.iloc[500,0]

'(Олег) Вы думаете что это просто программа которая решает за вас всю вашу работу? (показывает руками), но так ли? Если мы посмотрим вокруг, а именно, через призму ваших мыслей и интересов, то увидим насколько глубоко вы можете проникнуть своими рассуждениями по этому поводу? И как высоко может подниматься ваша самооценка когда у вас будет такое превосходное знание вопроса. Так вот ваш разум подсказывает вам этот важный ответ....И тогда все остальные слова начинают звучать для нас очень заманчиво..... Рекомендовано к прочтению! Оригинал взят ув Рекомендованно к чтению!"Это - не только про тебя".Короткий рассказ из сборника рассказов о войне "Берег" / Составитель Александр Куряшин/ Под ред. А..Четверикова; Художник Юрий Пасечников.- М.: Правда СССР, 1967 г., 128 стр.; 2500 экз.(на русском языке). Внимание!!! Всем любителям творчества Михаила Щербакова настоятельно рекомендуем ознакомиться со следующим его материалом   \xa0Собрание сочинений МАММЕТРОПОЛЯ СОННЫХ ПЫЛИЦА, или ОБУЗА ПОЛИТЗАК

In [63]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7471 entries, 0 to 7470
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    6981 non-null   object 
 1   model   6960 non-null   object 
 2   length  6981 non-null   float64
dtypes: float64(1), object(2)
memory usage: 175.2+ KB


In [35]:
df = pd.read_csv("C:/Users/KseniaLuschevaext/Documents/thesis/gen_texts_6981.csv")
df.drop('Unnamed: 0', inplace=True, axis=1)
df_n = pd.concat([df,df_all], ignore_index=True)

In [36]:
df_trim = df_n.copy(deep=True)
df_trim['text'] = df_trim['text'].str.replace(r'[\n\t\\t]', ' ', regex=True)

In [37]:
df_trim['text'] = df_trim['text'].str.split('НАЧАЛО ТЕКСТА').str[-1]

In [38]:
df_trim['text'] = df_trim['text'].apply(lambda x: x[2:] if str(x).startswith("] ") else x)

In [39]:
df_trim.dropna(subset=['text'], inplace=True)
df_trim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9916 entries, 0 to 9915
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    9916 non-null   object 
 1   model   9895 non-null   object 
 2   length  9916 non-null   float64
dtypes: float64(1), object(2)
memory usage: 232.5+ KB


In [40]:
df_trim.to_csv('gen_texts_9916.csv')

In [41]:
df_trim.head()

,text,model,length
0,Сегодня мы обсудим загадочные и малоизученные ...,IlyaGusev/saiga_yandexgpt_8b,437.0
1,Рассмотрев современные тенденции развития наук...,IlyaGusev/saiga_yandexgpt_8b,430.0
2,"### Содержание #### Введение В мире, где каж...",IlyaGusev/saiga_yandexgpt_8b,414.0
3,## Содержание статьи В данной статье будет ра...,IlyaGusev/saiga_yandexgpt_8b,480.0
4,Сегодня мы наблюдаем за одной из самых значите...,IlyaGusev/saiga_yandexgpt_8b,353.0


In [42]:
percent_distribution = df_trim['model'].value_counts(normalize=True) * 100
print(df_trim.info())
print(percent_distribution)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9916 entries, 0 to 9915
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    9916 non-null   object 
 1   model   9895 non-null   object 
 2   length  9916 non-null   float64
dtypes: float64(1), object(2)
memory usage: 232.5+ KB
None
model
ai-forever/rugpt3large_based_on_gpt2    38.534613
deepseek-chat                           21.374432
YandexGPT                               19.403739
GigaChat                                10.025265
IlyaGusev/saiga_yandexgpt_8b             4.567964
GigaChat-Pro                             3.759474
ai-forever/rugpt3small_based_on_gpt2     1.485599
GigaChat-Max                             0.848914
Name: proportion, dtype: float64
